9.13~10.15

## 9.13 인스턴스 생성 조절에 메타클래스 사용

싱글톤, 캐싱 등 기능 구현?
- 인스턴스 생성 못하도록

In [1]:
class NoInstances(type):
    def __call__(self, *args, **kwargs):
        raise TypeError("Can't instantiate directly")

In [2]:
class Spam(metaclass=NoInstances):
    @staticmethod
    def grok(x):
        print('Spam.grok')

In [3]:
Spam.grok(42) #스태틱 메소드를 호출할 순 있음

Spam.grok


In [4]:
s = Spam() #일반적인 방법으로 인스턴스 생성은 못함

TypeError: Can't instantiate directly

싱클톤 패턴 구현 = 오직 하나의 인스턴스 생성만 허용하는 클래스

In [7]:
#이렇게 하면 오직 하나의 인스턴스만 생성됨
class Singleton(type):
    def __init__(self, *args, **kwargs):
        self.__instance = None
        super().__init__(*args, **kwargs)
        
    def __call__(self, *args, **kwargs):
        if self.__instance is None:
            self.__instance = super().__call__(*args, **kwargs)
            return self.__instance
        else:
            return self.__instance

In [8]:
class Spam(metaclass=Singleton):
    def __init__(self):
        print('Creating Spam')

In [10]:
a = Spam()

Creating Spam


In [11]:
b = Spam()

In [12]:
a is b

True

In [13]:
c = Spam()
a is c

True

* 메타클래스 사용 안하고 구현

In [ ]:
class _Spam:
    def __init__(self):
        print('Creating Spam')

_spam_instance = None
def Spam():
    global _spam_instance
    if _spam_instance is not None:
        return _spam_instance
    else:
        _spam_instance = _Spam()
        return _spam_instance

캐시 인스턴스 만들때

In [14]:
import weakref
class Cached(type):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.__cache = weakref.WeakValueDictionary()
        
    def __call__(self, *args):
        if args in self.__cache:
            return self.__cache[args]
        else:
            obj = super().__call__(*args)
            self.__cache[args] = obj
            return obj

In [15]:
class Spam(metaclass = Cached):
    def __init__(self, name):
        print('Creating Spam({!r})'.format(name))
        self.name = name

In [16]:
a = Spam('Guido')

Creating Spam('Guido')


In [17]:
b = Spam('foo')

Creating Spam('foo')


In [18]:
c = Spam('Guido') #캐시

In [19]:
a is b

False

In [20]:
a is c

True

## 9.17 클래스 코딩 규칙 강제

## 9.21 프로퍼티 메소드 중복 피하기
- 타입 확인 같은 작업 수행하는 프로퍼티를 반복적으로 정의하는 클래스 -> 단순화 + 중복 피하기

In [25]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
    @property
    def name(self):
        return self._name
    
    @name.setter
    def name(self, value):
        if not isinstance(value, str):
            raise TypeError('name must be a string')
        self._name = value
        
    @property
    def age(self):
        return self._age
    
    @age.setter
    def age(self, value):
        if not isinstance(value, int):
            raise TypeError('age must be an int')
        self._age = value

프로퍼티 생성 -> 프로퍼티 객체 반환

In [26]:
def typed_property(name, expected_type):
    storage_name = '_' + name
    
    @property
    def prop(self):
        return getattr(self, storage_name)
    
    @prop.setter
    def prop(self, value):
        if not isinstance(value, expected_type):
            raise TypeError('{} must be a {}'.fotmat(name, expected_type))
        setattr(self, storage_name, value)
    return prop


In [27]:
class Person:
    name = typed_property('name', str)
    age = typed_property('age', int)
    def __init__(self, name, age):
        self.name = name
        self.age = age
        

In [28]:
from functools import partial
String = partial(typed_property, expected_type = str)
Integer = partial(typed_property, expected_type = int)

class Person:
    name = String('name')
    age = Integer('age')
    def __init__(self, name, age):
        self.name = name
        self.age = age

## 9.25 파이썬 바이트 코드 디스어셈블
- 코드를 하위 레벨 바이트 코드로 disassemble해서 내부 동작성 분석
- 프로그램을 매우 낮은 하위 레벨까지 분석해야될 때 dis 모듈 유용

In [29]:
def countdown(n):
    while n > 0:
        print('T-minus', n)
        n -= 1
    print('Blastoff')


In [30]:
import dis
dis.dis(countdown)

  2           0 SETUP_LOOP              39 (to 42)
        >>    3 LOAD_FAST                0 (n)
              6 LOAD_CONST               1 (0)
              9 COMPARE_OP               4 (>)
             12 POP_JUMP_IF_FALSE       41

  3          15 LOAD_GLOBAL              0 (print)
             18 LOAD_CONST               2 ('T-minus')
             21 LOAD_FAST                0 (n)
             24 CALL_FUNCTION            2 (2 positional, 0 keyword pair)
             27 POP_TOP

  4          28 LOAD_FAST                0 (n)
             31 LOAD_CONST               3 (1)
             34 INPLACE_SUBTRACT
             35 STORE_FAST               0 (n)
             38 JUMP_ABSOLUTE            3
        >>   41 POP_BLOCK

  5     >>   42 LOAD_GLOBAL              0 (print)
             45 LOAD_CONST               4 ('Blastoff')
             48 CALL_FUNCTION            1 (1 positional, 0 keyword pair)
             51 POP_TOP
             52 LOAD_CONST               0 (None)
             

In [31]:
countdown.__code__.co_code #dis() 함수가 해석한 로우 바이트 코드

b"x'\x00|\x00\x00d\x01\x00k\x04\x00r)\x00t\x00\x00d\x02\x00|\x00\x00\x83\x02\x00\x01|\x00\x00d\x03\x008}\x00\x00q\x03\x00Wt\x00\x00d\x04\x00\x83\x01\x00\x01d\x00\x00S"

In [32]:
c = countdown.__code__.co_code
import opcode
opcode.opname[c[0]]

'SETUP_LOOP'

In [33]:
opcode.opname[c[3]]

'LOAD_FAST'

dis 모듈 바이트 코드를 프로그램적으로 쉽게 처리할 수 있는 함수 -> 제너레이터 사용

In [38]:
import opcode

def generate_opcodes(codebytes):
    extended_arg = 0
    i = 0
    n = len(codebytes)
    while i < n:
        op = codebytes[i]
        i += 1
        if op >= opcode.HAVE_ARGUMENT:
            oparg = codebytes[i] + codebytes[i+1]*256 + extended_arg
            extended_arg = 0
            i += 2
            if op == opcode.EXTENDED_ARG:
                extended_arg = oparg * 65536
                continue
        else:
            oparg = None
        yield (op, oparg)

In [39]:
for op, oparg in generate_opcodes(countdown.__code__.co_code):
    print(op, opcode.opname[op], oparg)

120 SETUP_LOOP 39
124 LOAD_FAST 0
100 LOAD_CONST 1
107 COMPARE_OP 4
114 POP_JUMP_IF_FALSE 41
116 LOAD_GLOBAL 0
100 LOAD_CONST 2
124 LOAD_FAST 0
131 CALL_FUNCTION 2
1 POP_TOP None
124 LOAD_FAST 0
100 LOAD_CONST 3
56 INPLACE_SUBTRACT None
125 STORE_FAST 0
113 JUMP_ABSOLUTE 3
87 POP_BLOCK None
116 LOAD_GLOBAL 0
100 LOAD_CONST 4
131 CALL_FUNCTION 1
1 POP_TOP None
100 LOAD_CONST 0
83 RETURN_VALUE None


## 10.4 모듈을 여러 파일로 나누기

mymodue.py 파일을 두 개로 나누고 파일마다 클래스 정의를 넣고 싶을 때

In [ ]:
#mymodule.py
class A:
    def name(self):
        print('A.spam')
class B(A):
    def bar(self):
        print('B.bar')

mymodule.py 파일을 mymodule 디렉터리로 치환 -> 디렉터리에 다음과 같은 파일 생성

- __ init__.py
- a.py
- b.py

In [ ]:
# a.py
class A:
    def spam(self):
        print('A.spam')

In [ ]:
# b.py
from .a import A
class B(A):
    def bar(self):
        print('B.bar')

In [ ]:
# __init__.py
from .a import A
from .b import B

In [ ]:
import mymodule
a = mymodule.A()
a.spam()

In [ ]:
b = mymodule.B()
b.bar()

## 10.8 패키지의 데이터 파일 읽기
- 패키지는 zip이나 .egg 파일로 설치되어 있어 일반적인 디렉터리의 파일과 좀 다름 -> open() 쓰면 오류 날수도

mypackage/
   - __ init__.py
   - somedata.dat
   - spam.py
   
   spam.py 파일이 somedata.dat 파일의 데이터 읽음

In [ ]:
# spam.py
import pkgutil
data = pkgutil.get_data(__package__, 'somedata.data') # 첫번째 인자 : 패키지 이름, 두번째 인자 : 파일의 상대적 경로 이름

pkgutil.get_data() 함수는 패키지가 어떻게 설치되었는지 어디에 설치된지 상관 없이 데이터 파일을 얻는 상위 레벨 도구로 사용됨

## 10.12 임포트 시 모듈 패치
기존 모듈 함수에 데코레이터를 적용 or 패치 (실제로 임포트되고 사용되었을 때만)

In [36]:
#postimport.py

import importlib
import sys
from collections import defaultdict

_post_import_hooks = defaultdict(list)

class PostImportFinder:
    def __init__(self):
        self._skip = set()
        
    def find_module(self, fullname, path=None):
        if fullname in self._skip:
            return None
        self._skip.add(fullname)
        return PostImportLoader(self)
class PostImportLoader:
    def __init__(self, finder):
        self._finder = finder
        
    def load_module(self, fullname):
        importlib.import_module(fullname)
        module = sys.modules[fullname]
        for func in _post_import_hooks[fullname]:
            func(module)
        self._finder._skip.remove(fullname)
        return module
    
def when_imported(fullname):
    def decorate(func):
        if fullname in sys.modules:
            func(sys.modules[fullname])
        else:
            _post_import_hooks[fullname].append(func)
        return func
    return decorate

sys.meta_path.insert(0, PostImportFinder())

In [ ]:
from postimport import when_imported

@when_imported('threading') #임포트할 때 실행할 처리 함수를 등록
def warn_threads(mod):
    print('Threads? ARe you crazy?')
    
import threading

@when_imported : 임포트할 때 실행할 처리 함수를 등록
2. 데코레이터는 이미 불러온 모듈이 없는지 sys.modules 확인
3. 있으면 핸들러 호출
4. 아니면 핸들러를 _post_import_hooks 딕셔너리 리스트에 추가